In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0-beta1
sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)
matplotlib 3.1.2
numpy 1.16.2
pandas 0.25.3
sklearn 0.22
tensorflow 2.0.0-beta1
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
path_to_file = tf.keras.utils.get_file(
    'shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
print(path_to_file)
text = open(path_to_file, "r").read()
print(len(text))
print(text[0:100])

C:\Users\A\.keras\datasets\shakespeare.txt
1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
# 1. generate vocab
# 2. build mapping  char -> id
# 3. data -> id data
# 4. 预测下一个字符的模型  abcd - > bcd<eos>

#### 1. generate vocab

In [5]:
vocab = sorted(set(text))
print(vocab)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


#### 2. build mapping  char <=> id

In [6]:
# enumerate() 会对每一个元素生成一个index,
char2idx = {char :idx for idx,char in enumerate(vocab) }
print(char2idx)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [7]:
idx2char = np.array(vocab)
print(idx2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


#### 3. data => id data

In [8]:
text_as_int = np.array([char2idx[c] for c in text])
print(text_as_int[0:100])

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59]


#### 4. make dataset

In [9]:
# 样本抽取,用dataset的方式
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
seq_dataset = char_dataset.batch(seq_length + 1, 
                                 drop_remainder = True)

for ch_id in char_dataset.take(2):
    print(ch_id, idx2char[ch_id.numpy()])

for seq_id in seq_dataset.take(2):
    print(seq_id)
#     print("".join(idx2char[seq_id.numpy()]))
    print(repr("".join(idx2char[seq_id.numpy()])))

tf.Tensor(18, shape=(), dtype=int32) F
tf.Tensor(47, shape=(), dtype=int32) i
tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1], shape=(101,), dtype=int32)
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49], shape=(101,), dtype=int32)
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


#### 5. dataset中的每个batch(0:100) 拆分成（0：99）和（1：100）

In [10]:
# 把字符集分配成输入和输出，作为训练样本
def split_input_target(id_text):
    """
    abcde -> abcd,bcde
    """
    return id_text[0:-1], id_text[1:]


In [11]:
seq_dataset = seq_dataset.map(split_input_target)

for item_input, item_output in seq_dataset.take(2):
    print(item_input.numpy())
    print(item_output.numpy())
    

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59]
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1]
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1]
[56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1 58
 53  1 42

#### 6. 模型建立

In [12]:
batch_size = 64
buffer_size = 10000

seq_dataset = seq_dataset.shuffle(buffer_size).batch(
    batch_size, drop_remainder = True
)

In [13]:
print(text[0:100])
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim, 
                               batch_input_shape=[batch_size, None]),
        keras.layers.LSTM(units = rnn_units,
                          recurrent_initializer='glorot_uniform',
                          stateful=True,
                          return_sequences = True),
        keras.layers.Dense(vocab_size)
    ])
    return model
model = build_model( vocab_size = vocab_size,
                     embedding_dim = embedding_dim,
                     rnn_units = rnn_units,
                     batch_size = batch_size)

model.summary()

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [14]:
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(labels, 
                                                      logits, 
                                                      from_logits = True)

model.compile(optimizer = "adam", loss = loss)

#### 8. 模型训练

In [15]:
output_dir = "./text_generation_lstm_checkpoints"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
checkpoint_prefix = os.path.join(output_dir, "ckpt_{epoch}")
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
#     monitor='val_loss',
#     verbose=0,
#     save_best_only=False,
    save_weights_only=True,
#     mode='auto',
#     save_freq='epoch',
#     load_weights_on_restart=False
)

epochs = 100

history = model.fit(seq_dataset, epochs = epochs,
                   callbacks= [checkpoint_callback])

Epoch 1/100
172/172 [==============================] - 13s 75ms/step - loss: 2.5805
Epoch 2/100
172/172 [==============================] - 11s 63ms/step - loss: 1.8759
Epoch 3/100
172/172 [==============================] - 11s 63ms/step - loss: 1.6287
Epoch 4/100
172/172 [==============================] - 11s 65ms/step - loss: 1.4950
Epoch 5/100
172/172 [==============================] - 11s 65ms/step - loss: 1.4122
Epoch 6/100
172/172 [==============================] - 11s 65ms/step - loss: 1.3511
Epoch 7/100
172/172 [==============================] - 11s 65ms/step - loss: 1.3005
Epoch 8/100
172/172 [==============================] - 11s 63ms/step - loss: 1.2533
Epoch 9/100
172/172 [==============================] - 11s 65ms/step - loss: 1.2075
Epoch 10/100
172/172 [==============================] - 11s 64ms/step - loss: 1.1631
Epoch 11/100
172/172 [==============================] - 11s 64ms/step - loss: 1.1226
Epoch 12/100
172/172 [==============================] - 11s 64ms/step - lo

172/172 [==============================] - 12s 68ms/step - loss: 0.2872
Epoch 97/100
172/172 [==============================] - 12s 70ms/step - loss: 0.2871
Epoch 98/100
172/172 [==============================] - 12s 69ms/step - loss: 0.2856
Epoch 99/100
172/172 [==============================] - 12s 70ms/step - loss: 0.2819
Epoch 100/100
172/172 [==============================] - 12s 71ms/step - loss: 0.2796


#### 9. 重载模型（利用checkpoint）

In [16]:
tf.train.latest_checkpoint(output_dir)

'./text_generation_lstm_checkpoints\\ckpt_100'

In [17]:

model2 = build_model(vocab_size = vocab_size,
                     embedding_dim = embedding_dim,
                     rnn_units = rnn_units,
                     batch_size = 1)
model2.load_weights(tf.train.latest_checkpoint(output_dir))
model2.build(tf.TensorShape([1,None]))

# start ch sequence A,
# A -> model -> b
# A.append(b) -> B
# B -> model-> c
# B.append(c) -> C
model2.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [23]:
def generate_text(model,start_string, num_generate = 1000):
    input_eval = [char2idx[ch] for ch in start_string]
    input_eval= tf.expand_dims(input_eval, 0)
    
    text_generated = []
    model.reset_states()
    
    # temperature > 1, 概率分布更平滑  随意
    # temperature < 1, 概率分布更陡峭  greedy
    temperature = 0.5
    
    for _ in range(num_generate):
        # 1.model inference ->predictions
        # 2.sample
        # 3.update input_eval
        
        # predictions : [batch_size, input_eval_len, vocab_size]
        predictions = model(input_eval)
        # temperature 作用
        # predictions : logits -> softmax -> prob
        # softmax: e^xi
        # eg: 4,2   e^4/(e^4 + e^2) = 0.88   e^2/(e^4 + e^2) = 0.12
        #     2,1   e^2/(e^1 + e^2) = 0.73   e^1/(e^1 + e^2) = 0.27
        predictions = predictions / temperature
        
        # predictions : [input_eval_len, vocab_size]   消掉batch_size的维度
        predictions = tf.squeeze(predictions, 0)
        # predicetd_ids : [input_eval_len, 1]
        # a b c -> b c d ,只用最后一个 predicetd_ids 就是d
        predicted_id = tf.random.categorical(
            predictions, num_samples = 1
        )[-1,0].numpy()
        text_generated.append(idx2char[predicted_id])
        # 为什么不在input_eval后面添加predicted_id，而是直接替换  => 为了防止低效
        # s, x -> rnn -> s', y  
        input_eval = tf.expand_dims([predicted_id],0)
    return start_string + "".join(text_generated)
new_text = generate_text(model2, "All: ")
print(new_text)


All: and my conceit ir
speak not to be the coll the war;
And what that is not yet standing 'tword, as this?

quite a bod of lord, within this work,
That thou didst cry all for the present death.
I fear, by thy spirit to him, I had rather my
brother; the bound of Edward's cruppe or two fortune tide thy eyes
Of noble essimation and well gractly,
iccass I see her; for once more leisure
In every scene so much well as she;
why, there's a word. Come, madam wife to her
And unrusted to our waters and so strong add
provoke me to my stuff.

Page:
I'll see you on his knowledge.

AUFIDIUS:
My liege, I cannot do thee now, I had put us it finds,
Even to the firm of s he did die brow
Thy knowle guard with God,
Doth hunt ble heir to the drunkard let him do it.

LADY CAPULET:
O heart, about me: these I would have take to put you to
no further.

VIRGILIA:
No, good madam,--

PAULINA:
For which their head saitors, suffer it
To see your grave to be your queen, let them have
That vowburse in this all?

Allo

In [24]:
new_text_1 = generate_text(model2, "First Citizen")
print(new_text_1)

First Citizen:
Nay, now I see thee shame to make you guard with heaven
Would buy think it for them to work, he has here.

GRUMIO:
Katharina, you, lords!
My child o' the silent hour of your couse,
Live me thy merry melt the prond knowledge
The ladely day, that he hath done me with her age.
Swere arm, we will be a judge; but I can read
waiting justice of his advirence; but it is George
Save me to chase in te that public day!

QUEEN ELIZABETH:
By Him that raised un?

SICINIUS:
Say you so?

AUFIDIUS:
No more!

COMINIUS:
You shall not be awhieved.

PAULINA:
I care not:
Good faith, I'll prove him.

DUKE VINCENTIO:
A hundred cast our good will but a lord!

GLOUCESTER:
Now is the wind that follow this forward!

First Servant:
My lord?

LEONTES:
The gods here did stand for consul, which nd thou
art a
nothing make the damal's place,
Drawn and be patiently to greet young Paris have been a
formen to the earth
Or either partyrous to me. Let me take thou move of thy followers
Are the very pity: mise

In [20]:
"""
100次 没有
All: good my lord, and so much leising kind Fruntage;
When Tyrreat exile her maid at Montague.
A coward, loving correction. When thou mayst nothing else
By patience and I know by men:
Who increase thou of my mateous daughter,
Yor nothing chide as double at your own.

KING RICHARD III:
Here come the Lady Bohemiage, but were still
door eat and a homeful dinner sound imprison'd;
But no more senators, and they sad
some success of power to speak again concrain,
Harl power and brothers: and they shall power
That thou art so least and respites from gentle seat,
Who do apparelt in the secret message
That seem like progar of that vengeance one!
O Friend, with mine, that torthy priest
Meant with us sweet and procedged strikes
Are as time to teach and services be contrary,
And eart--
Comm, death, for death my duty milder stones,
If it possesses on the queen's blood of them,
And spit at all; good sorrow he doth approve
As sweet as shunble love to assising solier:
When he so say, that I should keep to g
"""

"\n100次 没有\nAll: good my lord, and so much leising kind Fruntage;\nWhen Tyrreat exile her maid at Montague.\nA coward, loving correction. When thou mayst nothing else\nBy patience and I know by men:\nWho increase thou of my mateous daughter,\nYor nothing chide as double at your own.\n\nKING RICHARD III:\nHere come the Lady Bohemiage, but were still\ndoor eat and a homeful dinner sound imprison'd;\nBut no more senators, and they sad\nsome success of power to speak again concrain,\nHarl power and brothers: and they shall power\nThat thou art so least and respites from gentle seat,\nWho do apparelt in the secret message\nThat seem like progar of that vengeance one!\nO Friend, with mine, that torthy priest\nMeant with us sweet and procedged strikes\nAre as time to teach and services be contrary,\nAnd eart--\nComm, death, for death my duty milder stones,\nIf it possesses on the queen's blood of them,\nAnd spit at all; good sorrow he doth approve\nAs sweet as shunble love to assising solier:

In [21]:
"""
First Citizen:
Before some we will. I shall be your good voices this
verier'd here, that rights that honourably in
Time little to a fine sun.

CLARENCE:
I mut a deal of person mouth the vantage.
Thou liberty much lend a fork:
Therefore impose not me: the cannot choose.
Come Hastings, whence comes but well, sure the rest,
Revolt our fathers are forth from Deepet:
But let me have a copil-hamed perfect
lay know the royalty of a good duke of God!

GLOUCESTER:
Up, do not so, but yet metware
The Emperor of this place of intesters' sake.

GREMIO:
Warwick, think'st thou, Nattle, and thy change pronounce
Edmard Cominius to Bolingbroke's proud joy?
When, join'd togars drown day son?
The tiger'st undergeant to my wife,
I think there be right shops as bit with sword;
Which I have given away with thee and thy arms,
In this my leave of Bories and York and Sir Sheal'd for her lives
Before thy service and so look'd together:
The words of wondrous sun to Pradut Shall,
Obstant my woman.

BIONDELLO:
I took the can wa
"""

"\nFirst Citizen:\nBefore some we will. I shall be your good voices this\nverier'd here, that rights that honourably in\nTime little to a fine sun.\n\nCLARENCE:\nI mut a deal of person mouth the vantage.\nThou liberty much lend a fork:\nTherefore impose not me: the cannot choose.\nCome Hastings, whence comes but well, sure the rest,\nRevolt our fathers are forth from Deepet:\nBut let me have a copil-hamed perfect\nlay know the royalty of a good duke of God!\n\nGLOUCESTER:\nUp, do not so, but yet metware\nThe Emperor of this place of intesters' sake.\n\nGREMIO:\nWarwick, think'st thou, Nattle, and thy change pronounce\nEdmard Cominius to Bolingbroke's proud joy?\nWhen, join'd togars drown day son?\nThe tiger'st undergeant to my wife,\nI think there be right shops as bit with sword;\nWhich I have given away with thee and thy arms,\nIn this my leave of Bories and York and Sir Sheal'd for her lives\nBefore thy service and so look'd together:\nThe words of wondrous sun to Pradut Shall,\nObs